## Gerando os observaveis

In [ ]:
import numpy as np
Matrizes_Base = [np.identity(2), [[0, 1], [1, 0]], [[0, -1j], [1j, 0]], [[1, 0], [0, -1]]]
Matrizes_Base_1 = [np.identity(3), [[0, 1, 0], [1, 0, 1], [0, 1, 0]],
                 [[0, -1j, 0], [1j, 0, -1j], [0, 1j, 0]], [[1, 0, 0], [0, 0, 0], [0, 0, -1]]]

def criaVetor(N, SigmaNum, distancia, vizinhos):
    vetorRepresentaCadeia = list()
    for i in range(0, N):
        vetorRepresentaCadeiaAux = list()
        for j in range(0, N):
            vetorRepresentaCadeiaAux.append(0)
        vetorRepresentaCadeiaAux[i] = SigmaNum
        for k in range(0, vizinhos):
            if (i + distancia) >= N:
                vetorRepresentaCadeiaAux[i + distancia - N] = SigmaNum
            else:
                vetorRepresentaCadeiaAux[i + distancia] = SigmaNum
        vetorRepresentaCadeia.append(vetorRepresentaCadeiaAux)
    return vetorRepresentaCadeia
# print(criaVetor(8, 1, 2, 2))

def matriz_Soma(vetorRepresentaCadeia, MatrizesBase):
    import numpy
    matrizSoma = 0
    for i in range(0, len(vetorRepresentaCadeia)):
        matrizSomaAux = 1
        for j in range(0, len(vetorRepresentaCadeia[0])):
            matrizSomaAux = numpy.kron(matrizSomaAux, MatrizesBase[vetorRepresentaCadeia[i][j]])
        matrizSoma += matrizSomaAux
    return matrizSoma
# print(matriz_Soma(criaVetor(4, 1, 1, 1), Matrizes_Base))


def observables(N):
    listaVetoresObservaveis = list()
    for i in range(1, 4):
        for j in range(1,int(N/2)+1):
            listaVetoresObservaveis.append(criaVetor(N, i, j, 1)[0])
        matrizesObservaveis = list()
    for i in range(0, len(listaVetoresObservaveis)):
        matrizesObservaveis.append(matriz_Soma([listaVetoresObservaveis[i]], Matrizes_Base))
    return matrizesObservaveis



In [ ]:
# Resolvendo o modelo de ANNNI ( Spin-1/2 Axial Next-Nearest-Neighbour Ising )
Matrizes_Base = [np.identity(2), [[0, 1], [1, 0]], [[0, -1j], [1j, 0]], [[1, 0], [0, -1]]]
def hamiltoniano_ANNNI(J, k, g, first_neighbors, seconds_neighbors, field):
    H = -J*(first_neighbors - k*seconds_neighbors + g*field) # eq hamiltoniano
    eigenvalue, eigenvector = np.linalg.eig(H) #tirar autovalores e autovetores
    eig = eigenvalue.reshape(len(eigenvalue), 1) #reformatar os autovalores
    minimum = np.min(eig)
    index_of_minimum = np.where(eig == minimum)
    pos = index_of_minimum[0]
    bra = (eigenvector[:, pos[0]]).reshape(len(eigenvalue), 1)
    ket = (eigenvector[:, pos[0]]).reshape(1, len(eigenvalue))
    p = bra @ ket    #matriz densidade
    return p

#           Variando g e k
def main(N, Xo, Xf, Yo, Yf, passosX, passosY, Matrizes_Base, nomeArquivoSemExt): #cria função
    import time
    import numpy as np
    start_time = time.time()  #tempo inicial
    file1 = open(nomeArquivoSemExt+'.csv', "w") #criando arquivo
    K = (-(Xf-Xo))/passosX #define o ponto inicial em x
    observaveis = observables(N) #chama a função de observáveis
    matrizSomaSzMaisUm = matriz_Soma(criaVetor(N, 3, 1, 1), Matrizes_Base) #matriz para o hamiltoniano
    matrizSomaSzMaisDois = matriz_Soma(criaVetor(N, 3, 2, 1), Matrizes_Base) #matriz para o hamiltoniano
    matrizSxSozinho = matriz_Soma(criaVetor(N, 1, 0, 0), Matrizes_Base) #matriz para o hamiltoniano
    for c1 in range(0, passosX): #vai variar o eixo x
        print(c1) #escreve em que ponto o código esta pra ter noção de tempo
        K += (Xf-Xo)/passosX #define o passo em x
        g = (-(Yf-Yo))/passosY #define o ponto inicial em y
        for c2 in range(0, passosY): #vai variar o eixo y
            g += (Yf-Yo)/passosY #define o passo do eixo y
            hamiltoniano = hamiltoniano_ANNNI(1, K, g, matrizSomaSzMaisUm, matrizSomaSzMaisDois, matrizSxSozinho) #re_
            #Resolve o hamiltoniano e retorna a matriz densidade
            file1.write("%.2f" % K) #escreve os valores do eixo x
            file1.write(',') #virgula para tabular
            file1.write("%.2f" % g) #escreve os valores do eixo y
            for c3 in range(0, len(observaveis)): #roda os observáveis
                file1.write(',') #vírgula para tabular
                traco = np.trace(hamiltoniano @ observaveis[c3]).real #realizando as projeções para cada observável
                file1.write("%.8f" % traco) #escreve o resultado para cada observavel
            file1.write('\n') #pula a linha
    file1.close() #fecha o arquivo
    print("--- %s seconds ---" % ((time.time() - start_time))) #escreve o tempo
main(8, 0, 1, 0, 1, 100, 100, Matrizes_Base, 'oito_ANNNI')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
--- 1279.6170399188995 seconds ---


## Gerando as labels

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#                         ANNNI
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

# Rotulando os as transições de fase para o modelo de ANNNI, usando Ising e BKT


import numpy as np

label = open('labels_ANNNI.csv', "w")
for c in range(0, 100):
     k = 0.01 * c
     for c1 in range(0, 100):
         g = 0.01 * c1
         if 0 < k <= 0.5:
             g_ising = ((1 - k) / k)*(1 - np.sqrt((1 - 3 * k + 4 * (k ** 2))/(1 - k)))
             if g > g_ising:
                 label.write('2')
                 label.write('\n')
             else:
                 label.write('1')
                 label.write('\n')
         if 1/2 <= k <= 3/2:
             g_bkt = 1.05 * np.sqrt((k - 0.5)*(k - 0.1))
             if g > g_bkt:
                 label.write('2')
                 label.write('\n')
             else:
                 label.write('3')
                 label.write('\n')
label.close()



# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

In [ ]:
labels = pd.read_csv('/content/labels_ANNNI.csv')
labels.head()

,1
0,1
1,1
2,1
3,1
4,1


In [ ]:
annni = pd.read_csv('/content/oito_ANNNI (1).csv')
annni

,0.00,0.00.1,0.00000000,0.00000000.1,0.00000000.2,0.00000000.3,0.00000000.4,0.00000000.5,0.00000000.6,0.00000000.7,1.00000000,1.00000000.1,1.00000000.2,1.00000000.3
0,0.00,0.01,0.000037,0.000025,0.000025,0.000025,-0.000013,-0.000000e+00,-0.000000,-0.000000,0.999975,0.999975,0.999975,0.999975
1,0.00,0.02,0.000150,0.000100,0.000100,0.000100,-0.000050,-0.000000e+00,-0.000000,-0.000000,0.999900,0.999900,0.999900,0.999900
2,0.00,0.03,0.000338,0.000225,0.000225,0.000225,-0.000113,-1.000000e-08,-0.000000,-0.000000,0.999775,0.999775,0.999775,0.999775
3,0.00,0.04,0.000600,0.000400,0.000400,0.000400,-0.000200,-4.000000e-08,-0.000000,-0.000000,0.999600,0.999600,0.999600,0.999600
4,0.00,0.05,0.000938,0.000626,0.000625,0.000625,-0.000313,-1.000000e-07,-0.000000,-0.000000,0.999375,0.999374,0.999374,0.999374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,0.99,0.95,0.534972,0.651396,0.506476,0.543271,-0.161274,2.838816e-01,0.026418,-0.089540,0.068674,-0.560639,-0.033464,0.478833
9995,0.99,0.96,0.543571,0.658351,0.514936,0.550733,-0.163256,2.845253e-01,0.026796,-0.088900,0.070480,-0.552794,-0.034402,0.469152
9996,0.99,0.97,0.551982,0.665100,0.523221,0.558034,-0.165154,2.850556e-01,0.027156,-0.088233,0.072254,-0.545090,-0.035322,0.459653
9997,0.99,0.98,0.560205,0.671647,0.531330,0.565174,-0.166969,2.854778e-01,0.027498,-0.087541,0.073996,-0.537528,-0.036222,0.450340


In [ ]:
annni.columns = ['g','k','feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'feature_9',
 'feature_10',
 'feature_11',
 'feature_12']

In [ ]:
annni

,g,k,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12
0,0.00,0.01,0.000037,0.000025,0.000025,0.000025,-0.000013,-0.000000e+00,-0.000000,-0.000000,0.999975,0.999975,0.999975,0.999975
1,0.00,0.02,0.000150,0.000100,0.000100,0.000100,-0.000050,-0.000000e+00,-0.000000,-0.000000,0.999900,0.999900,0.999900,0.999900
2,0.00,0.03,0.000338,0.000225,0.000225,0.000225,-0.000113,-1.000000e-08,-0.000000,-0.000000,0.999775,0.999775,0.999775,0.999775
3,0.00,0.04,0.000600,0.000400,0.000400,0.000400,-0.000200,-4.000000e-08,-0.000000,-0.000000,0.999600,0.999600,0.999600,0.999600
4,0.00,0.05,0.000938,0.000626,0.000625,0.000625,-0.000313,-1.000000e-07,-0.000000,-0.000000,0.999375,0.999374,0.999374,0.999374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,0.99,0.95,0.534972,0.651396,0.506476,0.543271,-0.161274,2.838816e-01,0.026418,-0.089540,0.068674,-0.560639,-0.033464,0.478833
9995,0.99,0.96,0.543571,0.658351,0.514936,0.550733,-0.163256,2.845253e-01,0.026796,-0.088900,0.070480,-0.552794,-0.034402,0.469152
9996,0.99,0.97,0.551982,0.665100,0.523221,0.558034,-0.165154,2.850556e-01,0.027156,-0.088233,0.072254,-0.545090,-0.035322,0.459653
9997,0.99,0.98,0.560205,0.671647,0.531330,0.565174,-0.166969,2.854778e-01,0.027498,-0.087541,0.073996,-0.537528,-0.036222,0.450340


In [ ]:
annni['label'] = labels

In [ ]:
annni.to_csv('oito_sitios.csv')